<a href="https://colab.research.google.com/github/MiguelAngel-ht/Machine_Learning_Models/blob/main/CHAT_BOT__using_SPACY___.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [1]:
import json
import re
import random

In [2]:
import spacy
import en_core_web_sm

# **Functions**

In [4]:
x_json = """{"data":[
    {
    "response_type": "greeting",
    "user_input": ["hello", "hi", "hey"],
    "bot_response": "Hey there!",
    "required_words": []
  },
  {
    "response_type": "greeting",
    "user_input": ["see you", "goodbye", "bye", "quit", "q"],
    "bot_response": "See you later!",
    "required_words": []
  },
  {
    "response_type": "greeting",
    "user_input": ["nice", "to", "meet", "you"],
    "bot_response": "The pleasure is all mine!",
    "required_words": ["nice", "meet", "you"]
  },
  {
    "response_type": "greeting",
    "user_input": ["nice", "good", "i love you", "excellent", "wow", "amazing", "beautiful",
                   "pretty", "handsome", "beautifull"],
    "bot_response": "You are a nice person :)",
    "required_words": []
  }"""

In [5]:
def phase3(text):
  res = ''
  nlp = en_core_web_sm.load()
  doc = nlp(text)
  for ent in doc.ents:
    if ent.label_ == 'ORG':
      res = '. I don’t work for ' + str(ent.text) + "."
      break
    elif ent.label_ == 'GPE':
      res = ". I’ve never been to " + ent.text + "."
      break
    else:
      for nc in doc.noun_chunks:
        res = " anything about " + nc.text + "."
        break
  return res

In [6]:
def random_string(input_string):
  try:
    response = "Sorry, I don't know about" + phase3(input_string)
  except:
    response = "Sorry, I don't know about it."
  return response

In [7]:
def load_answers():
  with open('answers.txt', 'r') as f:
      answers = [line.splitlines() for line in f]
  return answers

In [9]:
def load_questions():  
  with open('questions.txt', 'r') as f:
      strings = [line.splitlines() for line in f]
      questions = [line[0].split() for line in strings]
      for i, quest in enumerate(questions):
        for j, word in enumerate(quest):
          questions[i][j] = word.lower()          
  return questions

In [10]:
def load_requireds():  
  with open('required.txt', 'r') as f:
      strings = [line.splitlines() for line in f]
      requireds = [line[0].split() for line in strings]
      for i, quest in enumerate(requireds):
        for j, word in enumerate(quest):
          requireds[i][j] = word.lower()          
  return requireds

In [11]:
def load_json(answers, questions, requireds, x_json):
    
  x = ''
  for i in range(len(questions)):
      aux = ",{"
      aux += """ "response_type" : "question""" + str(i) + """","""
      aux += """ "user_input" : """ + str(questions[i]) + ""","""
      aux += """ "bot_response" : " """ + str(answers[i][0]) + """","""
      aux += """ "required_words" : """ + str(requireds[i]) + """}"""
      x += aux
  
  x_json += x + ']}'
  X_json = x_json.replace("\'",'"')
  return X_json

In [19]:
answers = load_answers()
questions = load_questions()
requireds = load_requireds()
X_json = load_json(answers, questions, requireds, x_json)

response_data = json.loads(X_json)
response_data = response_data['data']

def get_response(input_string):
    split_message = re.split(r'\s+|[,;!.-]\s*', input_string.lower())
    score_list = []

    # Check all the responses
    for response in response_data:
        response_score = 0
        required_score = 0
        required_words = response["required_words"]

        # Check if there are any required words
        if required_words:
            for word in split_message:
                if word in required_words:
                    required_score += 1

        # Amount of required words should match the required score
        if required_score == len(required_words):
            # Check each word the user has typed
            for word in split_message:
                # If the word is in the response, add to the score
                if word in response["user_input"]:
                    response_score += 1

        # Add score to list
        score_list.append(response_score)
        # Debugging: Find the best phrase

    # Find the best response and return it if they're not all 0
    best_response = max(score_list)
    response_index = score_list.index(best_response)

    # Check if input is empty
    if input_string == "":
        return "Please type something so we can chat :("

    # If there is no good response, return a random one.
    if best_response != 0:
        return response_data[response_index]["bot_response"]

    return random_string(input_string)

# **Main**

In [13]:
# TALK TO THE USER

def chat():
  # INICIO
  print("Hello! I know stuff about chat bots.", end="")
  print("\nWhen you're done talking, just say 'goodbye','quit' or 'q' \n")
  while True:
    user_input = input("You: ")              # GET INPUT
    response = get_response(user_input)
    print("Bot:", response)       # RETURN ANSWER

    if response == "See you later!":     # BREAK CONVERSATION 
      break

In [ ]:
chat()